# Chess-Bot

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment

In [2]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/wumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/wumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [3]:
train, val = cb.dataset.pos2vec_dataset()

In [4]:
pos2vec = cb.model.train_pos2vec(train, val)

Training layer 1/4
40000/40000 [==============================] - 72s 2ms/step - loss: 0.3130 - val_loss: 0.1196 - lr: 0.0050
Training layer 2/4
40000/40000 [==============================] - 71s 2ms/step - loss: 0.1093 - val_loss: 0.1046 - lr: 0.0050
Training layer 3/4
40000/40000 [==============================] - 78s 2ms/step - loss: 0.1066 - val_loss: 0.1040 - lr: 0.0050
Training layer 4/4
40000/40000 [==============================] - 80s 2ms/step - loss: 0.1060 - val_loss: 0.1041 - lr: 0.0050
Pos2Vec weights loading from /home/wumphlett/personal/Chess-Bot/models/pos2vec.h5


In [16]:
import numpy as np

ex = np.reshape(train[0][0][0], (1, 773))

print(train[0][0][0].shape)
print("ex")
print(ex)

print("enc")
print(pos2vec.predict(ex)[0])

(773,)
ex
[[False False False False False False False False False  True False False
  False  True False False  True False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False Fa

## DeepChess

In [33]:
train, val = cb.dataset.deepchess_dataset()

In [38]:
deepchess = cb.model.train_deepchess(train, val)

Pos2Vec weights loading from /home/wumphlett/personal/Chess-Bot/models/pos2vec.h5
20000/20000 [==============================] - 47s 2ms/step - loss: 0.5566 - accuracy: 0.6887 - val_loss: 0.4562 - val_accuracy: 0.7750 - lr: 0.0100


## Evaluation

In [45]:
deepchess = cb.model.get_deepchess()
deepchess.evaluate(train)

DeepChess weights loading from /home/wumphlett/personal/Chess-Bot/models/deepchess.h5
20000/20000 [==============================] - 44s 2ms/step - loss: 0.4753 - accuracy: 0.7592


[0.4753116965293884, 0.7591639757156372]

In [ ]:
deepc

## Sandbox